In [1]:
"""
The design of this comes from here:
http://outlace.com/Reinforcement-Learning-Part-3/
"""

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import RMSprop
from keras.layers.recurrent import LSTM
from keras.callbacks import Callback

# Adding this per a suggestion by Tim Kelch.
# https://medium.com/@trkelch/this-post-is-great-possibly-the-best-tutorial-explanation-ive-found-thus-far-cf78886b5378#.w473ywtbw


class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))


def neural_net(num_sensors, params, load=''):
    model = Sequential()

    # First layer.
    model.add(Dense(
        params[0], kernel_initializer='lecun_uniform', input_shape=(num_sensors,)
    ))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    # Second layer.
    model.add(Dense(params[1], kernel_initializer='lecun_uniform'))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    # Output layer.
    model.add(Dense(4, kernel_initializer='lecun_uniform'))
    model.add(Activation('linear'))

    rms = RMSprop()
    model.compile(loss='mse', optimizer=rms)

    if load:
        model.load_weights(load)

    return model


def lstm_net(num_sensors, load=False):
    model = Sequential()
    model.add(LSTM(
        output_dim=512, input_dim=num_sensors, return_sequences=True
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(output_dim=512, input_dim=512, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim=3, input_dim=512))
    model.add(Activation("linear"))
    model.compile(loss="mean_squared_error", optimizer="rmsprop")

    return model

Using Theano backend.


In [1]:
from carGame import carGame
import numpy as np
import random
import csv
from neuralNetwork import neural_net, LossHistory
import os.path
import timeit

NUM_INPUT = 2
GAMMA = 0.9  # Forgetting.
TUNING = False  # If False, just use arbitrary, pre-selected params.


def train_net(model, params):

    filename = params_to_filename(params)

    observe = 1000  # Number of frames to observe before training.
    epsilon = 1
    train_frames = 10000  # Number of frames to play.
    batchSize = params['batchSize']
    buffer = params['buffer']

    # Just stuff used below.
    max_car_distance = 0
    car_distance = 0
    t = 0
    data_collect = []
    replay = []  # stores tuples of (S, A, R, S').

    loss_log = []

    # Create a new game instance.
    game_state = carGame()

    # Get initial state by doing nothing and getting the state.
    _, state = game_state.update(0)

    # Let's time it.
    start_time = timeit.default_timer()

    # Run the frames.
    while t < train_frames:

        t += 1
        car_distance += 1

        # Choose an action.
        if random.random() < epsilon or t < observe:
            action = np.random.randint(0, 3)  # random
        else:
            # Get Q values for each action.
            qval = model.predict(state, batch_size=1)
            action = (np.argmax(qval))  # best

        # Take action, observe new state and get our treat.
        reward, new_state = game_state.update(action)

        # Experience replay storage.
        replay.append((state, action, reward, new_state))

        # If we're done observing, start training.
        if t > observe:

            # If we've stored enough in our buffer, pop the oldest.
            if len(replay) > buffer:
                replay.pop(0)

            # Randomly sample our experience replay memory
            minibatch = random.sample(replay, batchSize)

            # Get training values.
            X_train, y_train = process_minibatch(minibatch, model)

            # Train the model on this batch.
            history = LossHistory()
            model.fit(
                X_train, y_train, batch_size=batchSize,
                nb_epoch=1, verbose=0, callbacks=[history]
            )
            loss_log.append(history.losses)

        # Update the starting state with S'.
        state = new_state

        # Decrement epsilon over time.
        if epsilon > 0.1 and t > observe:
            epsilon -= (1/train_frames)

        # We died, so update stuff.
        if reward == -1000:
            print('crash reported');
            # Log the car's distance at this T.
            data_collect.append([t, car_distance])

            # Update max.
            if car_distance > max_car_distance:
                max_car_distance = car_distance

            # Time it.
            tot_time = timeit.default_timer() - start_time
            fps = car_distance / tot_time

            # Output some stuff so we can watch.
            print("Max: %d at %d\tepsilon %f\t(%d)\t%f fps" %
                  (max_car_distance, t, epsilon, car_distance, fps))

            # Reset.
            car_distance = 0
            start_time = timeit.default_timer()

        # Save the model every 25,000 frames.
        if t % 250 == 0:
            model.save_weights('saved-models/' + filename + '-' +
                               str(t) + '.h5',
                               overwrite=True)
            print("Saving model %s - %d" % (filename, t))

    # Log results after we're done all frames.
    log_results(filename, data_collect, loss_log)


def log_results(filename, data_collect, loss_log):
    # Save the results to a file so we can graph it later.
    with open('results/sonar-frames/learn_data-' + filename + '.csv', 'w') as data_dump:
        wr = csv.writer(data_dump)
        wr.writerows(data_collect)

    with open('results/sonar-frames/loss_data-' + filename + '.csv', 'w') as lf:
        wr = csv.writer(lf)
        for loss_item in loss_log:
            wr.writerow(loss_item)


def process_minibatch(minibatch, model):
    """This does the heavy lifting, aka, the training. It's super jacked."""
    X_train = []
    y_train = []
    # Loop through our batch and create arrays for X and y
    # so that we can fit our model at every step.
    for memory in minibatch:
        # Get stored values.
        old_state_m, action_m, reward_m, new_state_m = memory
        # Get prediction on old state.
        old_qval = model.predict(old_state_m, batch_size=1)
        # Get prediction on new state.
        newQ = model.predict(new_state_m, batch_size=1)
        # Get our best move. I think?
        maxQ = np.max(newQ)
        y = np.zeros((1, 3))
        y[:] = old_qval[:]
        # Check for terminal state.
        if reward_m != -500:  # non-terminal state
            update = (reward_m + (GAMMA * maxQ))
        else:  # terminal state
            update = reward_m
        # Update the value for the action we took.
        y[0][action_m] = update
        X_train.append(old_state_m.reshape(NUM_INPUT,))
        y_train.append(y.reshape(3,))

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train


def params_to_filename(params):
    return str(params['nn'][0]) + '-' + str(params['nn'][1]) + '-' + \
            str(params['batchSize']) + '-' + str(params['buffer'])


def launch_learn(params):
    filename = params_to_filename(params)
    print("Trying %s" % filename)
    # Make sure we haven't run this one.
    if not os.path.isfile('results/sonar-frames/loss_data-' + filename + '.csv'):
        # Create file so we don't double test when we run multiple
        # instances of the script at the same time.
        open('results/sonar-frames/loss_data-' + filename + '.csv', 'a').close()
        print("Starting test.")
        # Train.
        model = neural_net(NUM_INPUT, params['nn'])
        train_net(model, params)
    else:
        print("Already tested.")


if __name__ == "__main__":
    if TUNING:
        param_list = []
        nn_params = [[164, 150], [256, 256],
                     [512, 512], [1000, 1000]]
        batchSizes = [40, 100, 400]
        buffers = [10000, 50000]

        for nn_param in nn_params:
            for batchSize in batchSizes:
                for buffer in buffers:
                    params = {
                        "batchSize": batchSize,
                        "buffer": buffer,
                        "nn": nn_param
                    }
                    param_list.append(params)

        for param_set in param_list:
            launch_learn(param_set)

    else:
        nn_param = [164, 150]
        params = {
            "batchSize": 100,
            "buffer": 50000,
            "nn": nn_param
        }
        model = neural_net(NUM_INPUT, nn_param)
        train_net(model, params)

Using Theano backend.
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(164, input_shape=(2,), kernel_initializer="lecun_uniform")`
  params[0], init='lecun_uniform', input_shape=(num_sensors,)
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_initializer="lecun_uniform")`
  model.add(Dense(params[1], init='lecun_uniform'))
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, kernel_initializer="lecun_uniform")`
  model.add(Dense(3, init='lecun_uniform'))


(+) Game initialized successfully
85.80357622332677
95.60788996983874
101.74603432907423
101.94067670590687
101.693736498801
107.00217949063489
107.17278145988149
121.9649443036625
122.15058443970204
122.33765325753802
114.13085623327642
127.5657939721024
127.76146984108257
127.95871812861974
121.17889657749716
143.00944515744786
143.18990202168442
127.63840565109766
148.42010793969618
133.4146817003066
153.6616281598012
153.85612928332822
154.05243026453863
140.87256860649947
171.72546547164666
171.9530559106868
172.18355024194096
172.41699018079584
172.65341678457577
172.8928701965582
173.13538935822936
173.38101168794543
173.62977272436217
173.88170573328063
153.1942116702002
179.4419031710263
179.7073461531273
160.21527338817353
165.46499121675632
203.78052177456865
204.0736364896778
204.37222395427213
173.03943597990354
210.05209641939484
210.36469315472274
210.68291661742444
-179.185096197025
-173.6215793664876
-168.06350650955125
-127.92501088585493
-162.12448961347255
-117.6725

91.05711354206684
91.34592405219468
-1000
crash reported
Max: 175 at 426	epsilon 1.000000	(175)	27.348856 fps
72.47907180192763
104.07884148361538
104.2797186282315
104.48578769414028
-1000
crash reported
Max: 175 at 431	epsilon 1.000000	(5)	24.366403 fps
103.26354857465338
103.45288272274988
103.64660452662103
77.58296249407694
108.28625529565079
82.83881627990081
87.44333420789324
-1000
crash reported
Max: 175 at 439	epsilon 1.000000	(8)	26.007630 fps
126.16794673504285
89.83356396420712
94.92001296240205
142.09884275512115
99.85445450029394
146.86523774314963
146.74422598345268
105.40771131249006
110.51262716992136
163.55913866680217
115.71312302279102
168.7212044905557
168.13111934216886
167.48780554171015
166.783924601494
-1000
crash reported
Max: 175 at 455	epsilon 1.000000	(16)	27.321412 fps
120.10503636503796
173.34141460962883
172.79477933754544
172.20381892221647
124.5666859897144
130.67894719653515
136.6893691754392
143.09189306434686
189.38660218296414
187.87258877899552
18

-49.145541960421625
-43.525651608151044
-87.40944506373776
-37.96377305985455
-81.88367253372286
-32.535407787051525
-26.842218439218154
-21.148029063221376
-68.9983635145689
-68.67978439571144
-68.37130938926717
-68.07248768792749
-67.7828932754936
-14.965874709032168
-10.603241073417362
-49.121345557085974
-48.877825056600265
-48.64019729302821
-5.215273388173525
-41.33498759186521
-0.30985393213503165
-36.543865021759174
-36.219149568238535
5.03281339419859
9.683320487960234
14.335399056743702
19.227745317954177
23.94929630165611
1.7829994864216516
2.1100528511919237
30.309766851626264
7.383488083246416
35.887026266326664
24.323243363104332
24.578049127098787
42.572099837060684
29.937086604196736
30.19190350510963
49.22290846459051
54.364436825102395
50.54703474289428
50.783316416374895
51.02170419430951
61.63141201514051
56.376249256752374
67.58255420992371
71.78168863537648
72.00265270161731
72.22553819901897
72.45034916039405
72.67708680576563
76.21470128211543
81.46082091294448


/Applications/anaconda/lib/python3.6/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


...

140.83238662042223
146.02029230207123
151.13069855211288
156.57095116691386
161.75026450502574
167.2589542929041
172.76485330069747
178.5183487341991
-175.73473569082003
-170.15165398110315
-164.32093665595028
-158.49116216397456
-202.29440487267976
-203.05610714765248
-203.7798423529396
-204.4681719170893
-156.0295921915135
-200.87332262595072
-152.16498505733196
-147.68835389115344
-143.3690462932786
-192.63058992465466
-139.1508455195025
-134.6011802522866
-188.13371375061593
-129.90593717496347
-125.06858282186248
-120.45722550910554
-115.56626988955452
-110.7354877019201
-105.61860540890943
-100.49983323344742
-144.55229484753085
-94.57783868126131
-89.38559856547178
-84.02127322470257
-78.58276716956523
-73.13440598182699
-68.58276716956524
-64.0212732247026
-59.38559856547182
-54.57783868126131
Saving model 164-150-100-50000 - 8500
-49.76551355370726
-49.51631177939053
-43.86158142836596
-38.99076481684532
-33.83228234535838
-28.739795291688043
-23.36621169280037
-18.21391114965

135.75826016436676
140.94686305397352
146.39237368003702
118.5009692374598
150.5721978039638
109.5594428881281
107.9974770973065
154.46715906138928
-1000
crash reported
Max: 697 at 8893	epsilon 0.210700	(253)	13.846766 fps
168.1340223063963
173.9077778098547
179.67639313745002
-174.74488129694225
-168.90918365114783
-163.07248693585296
-158.90918365114783
-154.74488129694225
-150.32360686254998
-146.09222219014532
-141.86597769360372
-137.3845555935016
-132.9052429229879
-128.11134196037202
-176.54984893344
-123.01603062799532
-118.16635469383934
-113.37410135669141
-108.26487474140086
-103.15360586678052
-97.98274815743552
-92.80696552449304
-127.30439952229568
-86.43894759928277
-80.96375653207352
-75.47584806258082
-70.96375653207353
-66.4389475992828
-61.8389471344298
-57.02589727076334
-52.207697972859066
-51.33512409162745
-51.16411056028656
-44.98163936884934
-46.13367380705859
-45.94936522417971
-37.758540601060034
-32.58202920779717
-27.4877438424672
-22.064711005941724
-16.89

-117.00252652819623
-111.93675383430278
-152.9959189220557
-106.03958215958599
-100.92850242282282
-95.81465799383543
-90.49780599638339
-85.13419305691562
-79.76180233348705
-75.13419305691563
-70.4978059963834
-65.81465799383547
-60.92850242282281
-56.039582159586004
-51.11214830878025
-45.73519692702848
-45.61417081293718
-44.19385436247131
-39.00913380339451
-33.87081071038884
-28.465379346355327
-23.287527688912526
-17.84163397935893
-12.39723048230129
-6.7335191534162675
-1.0734133924689802
4.4207731275109765
10.135835503748016
15.851928154286954
20.135835503748044
24.42077312751099
28.926586607531032
33.26648084658371
37.60276951769868
42.15836602064112
46.71247231108751
51.53462065364471
56.12918928961118
60.9908661966055
65.80614563752872
70.89441839219026
75.98412732882637
81.12146019489035
86.26222575772358
91.59833161307313
96.99083497903926
102.39271347694603
119.9432506726783
105.77264764822071
110.39289216129545
115.06456820725188
119.93812397222489
124.81478667205226
11

-25.820724245525028
-19.993743985823365
-14.46281847204055
25.4123463975422
-8.718993973158076
-2.8767650703005767
1.2810060268419576
5.448738726860382
9.907884875501036
49.95281024708008
13.374125294382521
17.519113820476328
Saving model 164-150-100-50000 - 9750
21.950938298325493
26.384351815835885
31.057246051590287
35.44289577832933
40.059426966886974
44.72128019586774
49.61597931237985
54.50852298766841
59.3431517530787
64.17254689617462
69.36123339049324
74.49699698108216
79.63754090156813
84.49699698108216
89.36123339049324
94.17254689617462
99.3431517530787
104.5085229876684
109.61597931237985
114.72128019586775
120.05942696688697
125.44289577832934
131.05724605159025
90.88224107621113
135.44571032759825
140.9982448774387
146.55247275209155
152.3867782340334
158.22434383060096
163.75368437900988
169.57318750142718
119.45866609956073
175.2363583092738
179.4278021960362
-176.37071693189404
-171.886427331432
-167.7033982766004
-163.51723010830693
-159.05788812861763
-154.596924418

FileNotFoundError: [Errno 2] No such file or directory: 'results/sonar-frames/learn_data-164-150-100-50000.csv'

In [ ]:
"""
Once a model is learned, use this to play it.
"""

from carGame import carGame
import numpy as np
from neuralNetwork import neural_net

NUM_SENSORS = 2


def play(model):

    car_distance = 0
    game_state = carGame()

    # Do nothing to get initial.
    _, state = game_state.update(0)

    # Move.
    while True:
        car_distance += 1

        # Choose action.
        action = (np.argmax(model.predict(state, batch_size=1)))

        # Take action.
        _, state = game_state.update(action)

        # Tell us something.
        if car_distance % 1000 == 0:
            print("Current distance: %d frames." % car_distance)


if __name__ == "__main__":
    saved_model = 'saved-models/164-150-100-50000-10000.h5'
    model = neural_net(NUM_SENSORS, [164, 150], saved_model)
    play(model)

/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(164, input_shape=(2,), kernel_initializer="lecun_uniform")`
  params[0], init='lecun_uniform', input_shape=(num_sensors,)
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_initializer="lecun_uniform")`
  model.add(Dense(params[1], init='lecun_uniform'))
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, kernel_initializer="lecun_uniform")`
  model.add(Dense(3, init='lecun_uniform'))


(+) Game initialized successfully
4.846030199428071
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
